<a href="https://colab.research.google.com/github/Juanba98/SentimentAnalysisBERT/blob/main/sentimentAnalyisis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sentiment Analysis with [BERT](https://arxiv.org/pdf/1810.04805.pdf) (Pre-training of Deep Bidirectional Transformers for Language Understanding)**


# **1. Install and Import Dependencies** 

In [1]:
!pip install transformers &> /dev/null
print("Installing Transformers ...")

Installing Transformers ...


In [2]:
!apt install chromium-chromedriver &> /dev/null
!pip install selenium &> /dev/null
print("Installing Selenium for Web scraping ...")

Installing Selenium for Web scraping ...


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup #to extract our data
import re
from selenium import webdriver
import time
from selenium.webdriver.chrome.options import Options

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


# **2. Instantiate  Model**

In [4]:
#https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# **3. Encode and Calculate Sentiment**

In [5]:
tokens = tokenizer.encode("Not bad", return_tensors='pt')
print(f'Tokens: {tokens}')
print(f'Decode: {tokenizer.decode(tokens[0])}')

Tokens: tensor([[  101, 10497, 12428,   102]])
Decode: [CLS] not bad [SEP]


This model predicts the sentiment of the review as a number of stars (between 1 and 5).

In [6]:
result = model(tokens)
print(result.logits)
print(f'Number of stars: {int(torch.argmax(result.logits))+1}')

tensor([[-2.2001, -0.5841,  2.0563,  1.3172, -0.4261]],
       grad_fn=<AddmmBackward0>)
Number of stars: 3


# **4. Collect Reviews** 

In [7]:
chrome_options = Options()
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

url = "https://www.tripadvisor.es/Restaurant_Review-g187438-d21273712-Reviews-La_Caravana-Malaga_Costa_del_Sol_Province_of_Malaga_Andalucia.html"
driver.get(url)


time.sleep(10)
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()
regex = re.compile('partial_entry')
results = soup.find_all('p', {'class': regex})
reviews = [result.text for result in results]

In [8]:
reviews

['¡Si vienes a Málaga no te lo puedes perder!\nGrandísimo descubrimiento, es uno de los mejores sitios de la ciudad para comer. Cocina riquísima y de alta calidad. Además el trato es excelente, Álvaro y los chicos te harán sentir como en casa.\n¡Nos vemos...en la Caravana!Más',
 'Llevo comiendo aquí desde los primeros días que se abrió. Siempre hay buenísimo ambiente, los trabajadores son muy majos. Soy del barrio y los he visto crecer como local y cuando dieron con la tecla, desde ese momento no se ha bajado ni un poco...la calidad, ni de los productos ni de la ejecución de los platos.\n\nTienen los tercios de estrellas galicia mas fríos de Málaga, perfectos para acompañar la comida. \n\nEn definitiva, comida inspirada de diferentes culturas, en una mezcla de risas y buen rollo. \n\nRecomendable 100% si estás por Málaga.Más',
 'Soy muy fan de la mezcla de sabores que ofrecen los platos de este sitio. La calidad de las carnes, salsas y guarniciones hacen que vaya varias veces al mes, y

# **5. Load Review into DataFrame and Score** 

In [9]:
import numpy as np
import pandas as pd

In [10]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [11]:
df['review'].iloc[0]

'¡Si vienes a Málaga no te lo puedes perder!\nGrandísimo descubrimiento, es uno de los mejores sitios de la ciudad para comer. Cocina riquísima y de alta calidad. Además el trato es excelente, Álvaro y los chicos te harán sentir como en casa.\n¡Nos vemos...en la Caravana!Más'

In [12]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [13]:
#512 limit of tokens for the pipeline
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [14]:
df

,review,sentiment
0,¡Si vienes a Málaga no te lo puedes perder!\nG...,5
1,Llevo comiendo aquí desde los primeros días qu...,5
2,Soy muy fan de la mezcla de sabores que ofrece...,5
3,"Nos ha encantado descubrir platos nuevos, con ...",5
4,Almorzamos con un grupo de 12 personas despues...,5
5,"Pasamos una velada expléndida, la comida espec...",5
6,Ambiente especial y distinto... Gente de calle...,4
7,Después del campus pasé y tome una merienda qu...,5
8,Ese sabor no lo encuentras en otros lugares de...,5
9,Maravillosa ATENCION y la comida excelente. \n...,5


##Referential implementation:
- https://github.com/nicknochnack/BERTSentiment/blob/main/Sentiment.ipynbexamples/adversarial_patch/AdversarialPatch.ipynb
